# Supercharge Your Classes with Python `super()`

Let's consider two classes, `Square` and `Rectangle`. Their definition is very similar, and this can be a problem. If we introduced a bug in the definition of `area()`, we would have to fix it in *two* places.

In [2]:
class Square:
    def __init__(self, length):
        self.length = length
        
    def area(self):
        return self.length * self.length
    
    def perimeter(self):
        return 4 * self.length
    

class Rectangle:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        
    def area(self):
        return self.height * self.width
    
    def perimeter(self):
        return 2 * self.height+self.width
    
square = Square(3)
rectangle = Rectangle(2, 3)

print(square.area(), square.perimeter())
print(rectangle.area(), rectangle.perimeter())

9 12
6 7


 To avoid this duplication, we will define the `Square` class based on the `Rectangle` class via inheritance.

In [3]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)
    
square = Square(3)
rectangle = Rectangle(2, 3)

print(square.area(), square.perimeter())
print(rectangle.area(), rectangle.perimeter())

9 9
6 7


We need to redefine `__init__` as it requires only one argument for the `Square` class. If we do `dir()` on a square instance, we can see the `height` and `width` attributes.

In [3]:
dir(square)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'area',
 'height',
 'perimeter',
 'width']

Let's now define a `Cube` class, which also has the `surface_area()` and `volume()` methods. We can define this inheriting from `Square`. The code below shows two ways of doing the same thing.

In [4]:
class Cube(Square):
    def surface_area(self):
        face_area = self.area()
        return face_area * 6
    
    def volume(self):
        face_area = super().area()
        return face_area*self.length

`super()` called within a class gives access to the parent object. `super()` can also be called with parameters indicating the class and the object: `super(class, object)`. This form doesn't even have to be inside the object method.

Inside a class method, `super()` is a shortcut for `super(my_class, self)`. Let's add some methods to identify the class of an object

In [5]:
class Rectangle:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        
    def area(self):
        return self.height * self.width
    
    def perimeter(self):
        return 2 * self.height+self.width
    
    def what_am_i(self):
        return 'Rectangle'
    
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)
        
    def what_am_i(self):
        return 'Square'
    
class Cube(Square):
    def surface_area(self):
        face_area = self.area()
        return face_area * 6
    
    def volume(self):
        face_area = super().area()
        return face_area * self.length
    
    def what_am_i(self):
        return 'Cube'

We then call the `what_am_i()` method from classes higher and higher in the inheritance hierarchy.

In [6]:
cube = Cube(3)
print(cube.what_am_i())
print(super(Cube, cube).what_am_i())
print(super(Square, cube).what_am_i())

Cube
Square
Rectangle


We can make this more visible with the following modification to the `Cube` class.

In [7]:
class Cube(Square):
    def surface_area(self):
        face_area = self.area()
        return face_area * 6
    
    def volume(self):
        face_area = super().area()
        return face_area * self.length
    
    def what_am_i(self):
        return 'Cube'
    
    def family_tree(self):
        return self.what_am_i() + ' child of ' + super().what_am_i()
    
cube = Cube(3)
cube.family_tree()

'Cube child of Square'

## Multiple Inheritance

All these examples were based on single inheritance. Multiple inheritance is the process of inheriting from multiple classes into your new base class. Let's add a new shape, `Triangle` and a `RightPyramid` class that inherits simultaneously from `Triangle` and from `Square`. A right pyramid is formed by a square and four triangles. Note that `Triangle` does not inherit from anything (but `object`), and has not inherited method, like `area()` or `perimeter()`.

In [8]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height
        
    def area(self):
        return 0.5 * self.base * self.height
    
    def what_am_i(self):
        return 'Triangle'
    

class RightPyramid(Triangle, Square):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
        
    def what_am_i(self):
        return 'RightPyramid'
    
rightpyramid = RightPyramid(2, 4)
super(RightPyramid, rightpyramid).what_am_i()

'Triangle'

`Triangle` inherits from `Square`, therefore it has an `area()` and a `perimeter()` methods, but if we call them we get an error, since there is no `width` or `height` argument in the initializer.

In [9]:
rightpyramid.area()

AttributeError: 'RightPyramid' object has no attribute 'height'

The `super()` method returns the *first* parent. Since `Triangle` is referred before `Square` in the definition of the `RightPyramid` class, `Triangle` is the first parent.

We can see the class with the `.__class__` attribute and the **base classes** with `.__class__.__bases__`

In [10]:
print(rightpyramid.__class__)
print(rightpyramid.__class__.__bases__)

<class '__main__.RightPyramid'>
(<class '__main__.Triangle'>, <class '__main__.Square'>)


## MRO

A very important concept is the **MRO** (Method Resolution Order) is the order in which Python looks through the inheritance structure. This can be inspected via the `.__mro__` attribute.

In [10]:
RightPyramid.__mro__

(__main__.RightPyramid,
 __main__.Triangle,
 __main__.Square,
 __main__.Rectangle,
 object)

The MRO dictates how to look for methods. In multiple inheritance things can be complicated, as you may have methods with the same name in your inheriting classes. These name clashes can create confusion. The MRO dictates which method from which class is called. There are several possible workarounds: 

1. You could rename the methods so that there are no clashes
2. You can be careful about the order of inheritance when the class is defined. This solution is more opaque, as `class RightPyramid(Triangle, Square)` and `class RightPyramid(Square, Triangle)` call two different `area()` methods.
3. You can directly access the class to make a call, like `Square.area(self)`. This is the most explicit way.

Let's consider the following 5 classes.

In [11]:
class A:
    def __init__(self):
        print('A')  # Note that A has no parent apart from `object`
        super().__init__()
        
class B(A):
    def __init__(self):
        print('B')
        super().__init__()
        
class X:
    def __init__(self):
        print('X')
        super().__init__()
    
class Forward(B, X):
    def __init__(self):
        print('Forward')
        super().__init__()
        
class Backward(X, B):
    def __init__(self):
        print('Backward')
        super().__init__()

In [12]:
forward = Forward()

Forward
B
A
X


Here `Forward` goes back to `B`, which goes back to `A`, which has no parent, but then the other class `Forward` is inheriting from is invoked, and we have `X`, which is the end of the chain. Compare this with:

In [13]:
backward = Backward()

Backward
X
B
A


`X` has no parents, so `B` is called next, which inherits from `A`.

Let's leverage this complexity to redefine the `RightPyramid` and all the other classes so that they can receive keyword arguments. This works, but it is difficult to read.

```python
class Rectangle:
    def __init__(self, length, width, **kwargs):
        self.length = length
        self.width = width
        super().__init__(**kwargs)


class Square(Rectangle):
    def __init__(self, length, **kwargs):
        super().__init__(length=length, width=length, **kwargs)


class Triangle:
    def __init__(self, base, heigt, **kwargs):
            self.base = base
            self.heigt = heigt
            super().__init__(**kwargs)

            
class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height, **kwargs):
        self.base = base
        self.slant_height = slant_height
        kwargs['height'] = slant_height
        kwargs['length'] = base
        super().__init__(base=base, **kwargs)
```

## Mixins

One of the best ways of making sure you don't have problems is to create classes that do not have name clashes and that are as independent as possible. A pattern that is very common is a **Mixin**. A Mixin is a class that gets pulled into the inheritance hierarchy, but is not going to impact anything that inherits from it. For example, the `SurfaceAreaMixin` below, defines a `surface_area()` method, but has no expectations about construction. The only thing it requires is that the class using it contains a `surfaces` attribute. Below is an example of the `RightPyramid` class inheriting from a `SurfaceAreaMixin`.

Note that `SurfaceAreaMixin` does not have an `__init__` method. Its only purpose is to define a `surface_area()` method that can be used by the other classes. Note that the commented code is the original one. We have just modified it to be more compact. 

In [13]:
class SurfaceAreaMixin:
    def surface_area(self):
        surface_area = 0
        for surface in self.surfaces:
            surface_area += surface.area(self)
        return surface_area

class Cube(Square, SurfaceAreaMixin):
    def __init__(self, length):
        super().__init__(length)
        # self.surfaces = [Square, Square, Square, Square, Square, Square]
        self.surfaces = [Square] * 6

class RightPyramid(Square, Triangle, SurfaceAreaMixin):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
        self.height = slant_height
        self.length = base
        self.width = base
        # self.surfaces = [Square, Triangle, Triangle, Triangle, Triangle]
        self.surfaces = [Square] + [Triangle] * 4

        
cube = Cube(3)
cube.surface_area()

54